In [1]:
import datetime
import json
import time
import os
import datetime
import json
import time
from openai import AzureOpenAI
from dotenv import load_dotenv
import json
import copy
import textwrap

# cost per token for GPT4 8K
prompt_token_cost = 0.03/1000
generation_token_cost = 0.06/1000

In [2]:

# Load environment variables
load_dotenv()


True

Creating system prompt and user promot

In [3]:
model = os.getenv("MODEL")

user_messages = [
    "GPT-3 (Generative Pre-trained Transformer 3) - OpenAI's powerful language model capable of writing like a human",  
    "DALL-E - OpenAI's AI system that can create images from textual descriptions",  
    "वोडाफोन को आप शिकायत के लिए 199 पर कॉल कर सकते हैं। इसके अलावा आप 9820098200 पर भी फोन कर सकते हैं। इन दोनों नंबर पर प्रति 3 मिनट 50 पैस चार्ज लगेंगे।"
]

system_message = '''
You are an helpful AI assistant that identifies the language into "English" or "Non English".
###Important :
Do not add any additional information.
Make sure to complete all elements of the array'''


Sequential Execution

In [4]:
client = AzureOpenAI(
        api_version=os.getenv("API_VERSION"),
        azure_endpoint=os.getenv("AZURE_ENDPOINT"),
        api_key=os.getenv("API_KEY")
    )

responses = []

start_time = time.time()

for i, user_message in enumerate(user_messages):

    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message},
        ],
    )

    result = completion.choices[0].message.content
    responses.append(result)

end_time = time.time()
e2e_time = end_time - start_time

print(responses)
print("Time taken for execution : ", e2e_time)


['English', 'English', 'Non English']
Time taken for execution :  4.454633712768555


Batching Requests

In [5]:
def aoai_call(system_message,prompt, model):
    client = AzureOpenAI(
        api_version=os.getenv("API_VERSION"),
        azure_endpoint=os.getenv("AZURE_ENDPOINT"),
        api_key=os.getenv("API_KEY")
    )

    # start_time = time.time()

    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt},
        ],
    )

    # end_time = time.time()
    # e2e_time = end_time - start_time

    result=json.loads(completion.model_dump_json(indent=2))
    completion_text=result["choices"][0]["message"]["content"]

    return completion_text, e2e_time

aoai_call(system_message, 
          json.dumps([user_messages]), model)


('["English", "English", "Non English"]', 4.454633712768555)

Asynchronous call

In [5]:
import asyncio
import aiohttp
import os
from aiohttp import ClientSession

async def fetch(session, system_message, user_message):
    url = f'{os.getenv("AZURE_ENDPOINT")}/openai/deployments/gpt-4/chat/completions?api-version={os.getenv("API_VERSION")}'
    headers = {
        "Content-Type": "application/json",
        "api-key": os.getenv("API_KEY")
    }  
    data = {
        "model": "gpt-4",  # Adjust the model as needed
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message}
        ]
    }

    async with session.post(url, json=data, headers=headers) as response:
        return await response.json()

async def main(system_message, user_messages):
    api_call_batch_size = 16

    async with ClientSession() as session:
        tasks = []
        responses = []
        for i, user_message in enumerate(user_messages):
            task = asyncio.create_task(fetch(session, system_message, user_message))
            tasks.append(task)
            if len(tasks) >= api_call_batch_size: 
                responses.extend(await asyncio.gather(*tasks))
                tasks = []
        responses.extend(await asyncio.gather(*tasks))  # Process the last batch
        return responses

system_message = '''
You are an helpful AI assistant that identifies the language into "English" or "Non-English". 
###Important :
Do not add any additional information'''

start = time.time()
responses_async = await main(system_message, user_messages)
end = time.time()

run_time = end - start
print(f"Total time taken: {run_time} seconds")
# responses
response_content = []
for i in range(len(responses_async)):
    try:
        response_content.append(responses_async[i]["choices"][0]["message"]["content"])
    except:
        pass
print(response_content)
print(len(response_content))

Total time taken: 1.467106819152832 seconds
['AI', 'AI', 'SCIENCE', 'AI', 'SCIENCE', 'AI', 'AI', 'AI', 'SCIENCE', 'SCIENCE', 'AI', 'AI', 'AI', 'AI', 'SCIENCE', 'AI']
16


In [6]:
print(len(user_messages))

16


In [ ]:
s